In [1]:
%load_ext autoreload
%autoreload 2

In [23]:
import sys, os
from datetime import datetime, timedelta
print(os.getcwd())
sys.path.append('../')
from cognite.correlation.load_data import DataLoader
from cognite.correlation.correlation import columns_by_max_cross_correlation
from cognite.client import CogniteClient
import pandas as pd
import numpy as np
import json

client = CogniteClient(client_name='test-corr')
location = '../tests/hydrocyclones_tags.json'
tags_json = json.load(open(location, 'r'))


/Users/marcel/Documents/github/correlation/notebooks


/Users/marcel/.conda/envs/correlation-ranking/lib/python3.7/site-packages/cognite/client/_cognite_client.py:57: UserWarning: You are using version 1.0.0a33 of the SDK, however version 1.0.0a34 is available. Upgrade or set the environment variable 'COGNITE_DISABLE_PYPI_VERSION_CHECK' to suppress this warning.
  debug=debug,
/Users/marcel/.conda/envs/correlation-ranking/lib/python3.7/site-packages/ipykernel_launcher.py:12: UserWarning: Authenticated towards inferred project 'akerbp'. Pass project to the CogniteClient constructor or set the environment variable 'COGNITE_PROJECT' to suppress this warning.
  if sys.path[0] == '':


In [31]:
def read_dict_tree(tree):
    out = []
    for key, subtree in tree.items():
        if type(subtree) == dict:
            out += map(lambda x: key + '/' + x, read_dict_tree(subtree))
        else:
            out.append(key + '~' + subtree)
    return out
tag_list = read_dict_tree(tags_json)
tag_list = list(set(tag_list))
tags = pd.DataFrame({'description': list(map(lambda x: x.split('~')[0], tag_list)),
                     'CDF_TAG':         list(map(lambda x: x.split('~')[1], tag_list))})
print(tags.shape, '\n', tags.head())


(55, 2) 
                          description                       CDF_TAG
0    CHEMICALS/Scale inhibitor/Level              42LT4401/Y/1mMID
1               TURRET_CHOKE/BOA/PCV  16HCV0104/OutPosition/1mSAMP
2  TURRET_CHOKE/Kneler A + Bøyla/PCV  16HCV0204/OutPosition/1mSAMP
3   TURRET_CHOKE/East Kameleon/TM US              16TT0402/Y/1mMID
4             TURRET_CHOKE/BOA/PT US         16PST0140/MeasA/1mMID


In [32]:
# Retrieve data from CDF using ids
df = client.datapoints.retrieve_dataframe(
    # id=tags['id'].to_list(),
    external_id=tags['CDF_TAG'].to_list(),
    start=datetime(2019, 6, 30, 0, 0),
    end=datetime(2019, 7, 4, 0, 0),
    granularity='1m',
    aggregates=['interpolation']
)

print(df.head())

AssertionError: Time series identifier '20LV0220B/OutControl/1mMID' is duplicated in query

In [ ]:
# Rename columns
import re
df.columns = list(map(lambda x: re.sub('\|interpolation', '', x), df.columns))
print('\n'.join(map(str, enumerate(df.columns))))


In [ ]:
# Info about the dataframe
print(np.min(df.index.to_series().diff()))
print('Shape of df:', df.shape)


In [ ]:
import cognite.correlation
# Find correlation with optimal lag
lags = pd.timedelta_range(start=timedelta(minutes=-1), end=timedelta(), periods=60)
# print(lags)
correlate_to = df.columns[4]
correlations, cross = cognite.correlation.columns_by_max_cross_correlation(df, correlate_to, lags, return_cross_correlation_df=True)
print(correlations)


In [ ]:
import matplotlib.pyplot as plt
cross.plot()
plt.gcf().set_size_inches((20, 5))
